In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from os.path import join
from sklearn.preprocessing import LabelEncoder
import shutil, os, glob
from PIL import Image

In [ ]:
labels = pd.read_csv('labels.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
labels.groupby("breed").count().sort_values("id", ascending=False).head(10)

In [ ]:
labels.groupby("breed").count().sort_values("id", ascending=False).tail(10)

In [ ]:
breed = labels.breed.unique()

os.chdir("train")
os.mkdir("breed")
os.chdir("breed")

for breed_type in breed:
    os.mkdir(breed_type)
    print("Directory ", breed_type, " created")

In [ ]:
os.chdir("..")
os.getcwd()

In [ ]:
breed = labels.breed.unique()

os.mkdir("valid")
os.chdir("valid")
os.mkdir("breed")
os.chdir("breed")

for breed_type in breed:
    os.mkdir(breed_type)
    print("Directory ", breed_type, " created")

In [ ]:

os.chdir("..")
os.getcwd()

In [ ]:
os.chdir("..")
os.getcwd()

In [ ]:

x = labels.id
y = labels.breed

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
os.getcwd()

In [ ]:
os.getcwd()

In [ ]:
image_count_train = x_train.shape[0]
image_count_valid = x_valid.shape[0]

In [ ]:
image_count_train

In [ ]:
image_count_valid

In [ ]:
os.chdir("train")

for image in x_train.index:
    breed_type = labels.breed[image]
    path = "breed" + breed_type + "/"
    file = labels.id[image] + ".jpg"
    shutil.move(file, path)

In [ ]:
os.chdir("..")
os.getcwd()

In [ ]:
for image in x_valid.index:
    breed_type = labels.breed[image]
    path = "valid/breed/" + breed_type + "/"
    file = "train/" + labels.id[image] + ".jpg"
    shutil.move(file, path)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory="train/breed/",
    target_size=(input_size, input_size),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

In [ ]:

os.mkdir("test2")
shutil.move("test", "test2")
os.rename("test2", "test")

In [ ]:

input_size = 300
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory="train/breed/",
    target_size=(input_size, input_size),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory="valid/breed/",
    target_size=(input_size, input_size),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory="test/",
    target_size=(input_size, input_size),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling2D, Activation
from keras.optimizers import Adam, SGD, RMSprop, adadelta
from keras.models import Sequential, Model
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.utils import np_utils
from keras.models import model_from_json

In [ ]:
y_train = train_generator.classes
y_valid = valid_generator.classes
y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)

In [ ]:
inception_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(input_size,input_size,3))

In [ ]:
train_inception_v3 = inception_model.predict_generator(train_generator)

In [ ]:
np.save('train_inception_v3_300.npy', train_inception_v3)

In [ ]:
valid_inception_v3 = inception_model.predict_generator(valid_generator)
np.save('valid_inception_v3_300.npy', valid_inception_v3)

In [ ]:
train_inception_v3 = np.load('train_inception_v3_300.npy')

In [ ]:
valid_inception_v3 = np.load('valid_inception_v3_300.npy')

In [ ]:
valid_inception_v3 = np.load('valid_inception_v3_300.npy')

In [ ]:
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=train_inception_v3.shape[1:]))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(120, activation = 'softmax'))

In [ ]:
optimizer = RMSprop(lr = 0.0001, rho = 0.99)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(train_inception_v3, y_train, epochs=20, batch_size=batch_size, validation_data = (valid_inception_v3, y_valid))

In [ ]:
model_json = model.to_json()
with open("model_inception_v3.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_inception_v3.h5")
print("Saved model to disk")

In [ ]:
test_inception = inception_model.predict_generator(test_generator)

In [ ]:
np.save('test_inception_v3_300.npy', test_inception)

In [ ]:
test_inception = np.load('test_inception_v3_300.npy')

In [ ]:
prediction = model.predict(test_inception, verbose=1)

In [ ]:
json_file = open('model_inception_v3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model_inception_v3.h5")
print("Loaded model from disk")

In [ ]:
prediction = loaded_model.predict(test_inception, verbose=1)

In [ ]:
prediction = pd.DataFrame(prediction)

In [ ]:
prediction.to_csv('prediction_inception.csv')